In [1]:
class ESIOS:
    def __init__(self):
        #define API parameters
        pass
    
    def get_training_data(start_date, end_date, inputs):
        '''
        Download data from ESIOS API considering dates and specified demand.
        '''

        urls = {
            "demand": f"https://api.esios.ree.es/indicators/1775?type=3&start_date={start_date}&end_date={end_date}",
            "wind": f"https://api.esios.ree.es/indicators/1777?type=20&start_date={start_date}&end_date={end_date}",
            "solar": f"https://api.esios.ree.es/indicators/1779?type=3&start_date={start_date}&end_date={end_date}",
            "spot_price": f"https://api.esios.ree.es/indicators/602?type=3&start_date={start_date}&end_date={end_date}"
        }
        try:

            url = urls[inputs]

            # headers for the API
            headers = dict()
            headers['Accept'] = 'application/json; application/vnd.esios-api-v1+json'
            headers['Content-Type'] = 'application/json'
            headers['Host'] = 'api.esios.ree.es'
            headers['x-api-key'] = "27ac7b794ca773e7d1c6ea0f43adfd466abe7dbd6682b769ddd29cf25536c1d6"
            headers['Cookie'] = ''

            # connect to the API
            response = requests.get(url, headers=headers)

            if response.status_code == 200:
                data = response.json()
                # Process the data as needed
            else:
                print("Error: Request failed with status code", response.status_code)

            return data

        except Exception as e:

            print("could not retrieve data")
            print(e)
    

    def extract_values(data, call_data):
        '''
        Extracts values and time from the ESIOS data and returns a Pandas DataFrame.
        '''
        #values = [v["value"] for v in data["indicator"]["values"]]
        #times = [v["datetime"] for v in data["indicator"]["values"]]
        try:
            values = data["indicator"]["values"]
            df = pd.DataFrame(values)
            df = df[['value','datetime']]
            
            return df
        
        except Exception as e:
            print(f'No values could be extracted for {call_data}')
            print(e)
        # Create a DataFrame with 'time' and 'call_data' columns
        #df = pd.DataFrame({ 'time': times, call_data: values })

    def create_indicators_file ():

        if os.path.isfile("filename.txt"):
            print('The indicators file is already created!')


        else:
            try:    
                indicators = "https://api.esios.ree.es/indicators"
                headers = dict()
                headers['Accept'] = 'application/json; application/vnd.esios-api-v1+json'
                headers['Content-Type'] = 'application/json'
                headers['Host'] = 'api.esios.ree.es'
                headers['x-api-key'] = "27ac7b794ca773e7d1c6ea0f43adfd466abe7dbd6682b769ddd29cf25536c1d6"
                headers['Cookie'] = ''

                # connect to the API
                response = requests.get(indicators, headers=headers)

                if response.status_code == 200:
                    data = response.json()
                    # Process the data as needed
                else:
                    print("Error: Request failed with status code", response.status_code)

                df = pd.DataFrame(data["indicators"])
                df.to_csv('indicators.csv')
                print('Indicators file created succesfully!')
            except Exception as e:
                print('Indicators file could not be created')
                print(e)

    def connection_to_DB(df, call_data, year):
        '''
        Calls marginal_to_df and loads the df into a SQLite db
        '''

        try:


            conn = sqlite3.connect('Main_DB')
            c = conn.cursor()
            '''
            # Creates table to store marginalpdbc
            c.execute(f''''''CREATE TABLE IF NOT EXISTS {call_data} (
                    year number, 
                    month number, 
                    day number, 
                    hour number, 
                    price_ES number, 
                    price_PT number,
                    PRIMARY KEY (year, month, day, hour)
                    )
                    '''''')
            conn.commit()
            '''
            # Send dataframe to db
            df.to_sql(f'{call_data}', conn, if_exists='append', index=False,
                    method='multi', chunksize=1000)
            '''
            # Check for duplicates in the
            c.execute(f''''''
            DELETE FROM {call_data}
            WHERE ROWID NOT IN (
            SELECT MIN(ROWID) FROM {call_data}
            GROUP BY year, month, day, hour)
            '''''')
            '''
            print(f'{call_data} data uploaded succesfully for year {year}')



        except Exception as e:
            print("Could not upload the data to the DB")
            print(e)

In [2]:
start_date = "2019-01-01T00:00:00"

In [3]:
import pandas as pd

# Sample DataFrame with a column containing date-time values
data = {'DateTimeColumn': ['2019-01-01T00:00:00.000+01:00', '2020-02-02T12:30:00.000+02:00']}
df = pd.DataFrame(data)

# Splitting the DateTimeColumn into Date and Time components
df[['Date', 'Time']] = df['DateTimeColumn'].str.split('T', expand=True)

# Display the modified DataFrame
print(df)


                  DateTimeColumn        Date                Time
0  2019-01-01T00:00:00.000+01:00  2019-01-01  00:00:00.000+01:00
1  2020-02-02T12:30:00.000+02:00  2020-02-02  12:30:00.000+02:00


In [4]:
df['Time'] = df['Time'].str[:5]

In [5]:
df

,DateTimeColumn,Date,Time
0,2019-01-01T00:00:00.000+01:00,2019-01-01,00:00
1,2020-02-02T12:30:00.000+02:00,2020-02-02,12:30


In [6]:
gas_data['Delivery day'] = gas_data['Delivery day'].dt.strftime('%Y-%m-%d')

NameError: name 'gas_data' is not defined

In [ ]:
gas_data

,Delivery day,MIBGAS PVB Last Price Index Day-Ahead\n[EUR/MWh],MIBGAS PVB Average Price Index Day-Ahead\n[EUR/MWh],MIBGAS VTP Last Price Index Day-Ahead\n[EUR/MWh],MIBGAS VTP Average Price Index Day-Ahead\n[EUR/MWh],MIBGAS-ES Index\n[EUR/MWh],MIBGAS-PT Index\n[EUR/MWh],MIBGAS\nLNG-ES Index\n[EUR/MWh],MIBGAS\nAVB-ES Index\n[EUR/MWh]
0,2023-12-19,33.94,32.73,34.18,NaN,NaN,NaN,NaN,NaN
1,2023-12-18,31.06,30.90,31.23,NaN,33.53,32.50,31.37,31.88
2,2023-12-17,30.68,30.74,30.83,NaN,32.54,32.50,34.40,31.88
3,2023-12-16,30.68,30.74,30.83,NaN,31.92,32.50,30.80,31.88
4,2023-12-15,31.78,32.31,32.06,NaN,32.09,32.50,31.78,31.88
...,...,...,...,...,...,...,...,...,...
348,2023-01-05,58.00,57.45,NaN,NaN,58.74,65.00,56.60,70.00
349,2023-01-04,63.00,67.29,NaN,70.0,64.91,70.00,62.41,70.00
350,2023-01-03,60.70,57.73,NaN,NaN,67.00,73.00,62.43,67.00
351,2023-01-02,54.00,57.94,NaN,NaN,62.71,59.67,56.00,64.93


In [ ]:
import sqlite3
import pandas as pd

# Connect to the SQLite database (if the database doesn't exist, it will be created)
conn = sqlite3.connect('Main_DB')

# Create a cursor object to interact with the database
cursor = conn.cursor()

# Your SQL query
create_values_df = '''
SELECT * from gas_data g
JOIN demand d ON d.day = g.delivery_day
'''

# Read SQL query result into a DataFrame
df = pd.read_sql_query(create_values_df, conn)

# Close the connection
conn.close()

# Display the DataFrame
df

,delivery_day,index_price,value,day,hour
0,2019-12-31,11.80,19914.0,2019-12-31,04:00
1,2019-12-31,11.80,20196.0,2019-12-31,03:00
2,2019-12-31,11.80,20236.0,2019-12-31,05:00
3,2019-12-31,11.80,21021.0,2019-12-31,02:00
4,2019-12-31,11.80,21574.0,2019-12-31,06:00
...,...,...,...,...,...
178789,2022-01-01,99.00,23862.0,2022-01-01,19:00
178790,2022-01-01,99.00,24014.0,2022-01-01,22:00
178791,2022-01-01,99.00,24550.0,2022-01-01,20:00
178792,2022-01-01,99.00,24947.0,2022-01-01,21:00


In [ ]:
import sqlite3
import pandas as pd

# Connect to the SQLite database (if the database doesn't exist, it will be created)
conn = sqlite3.connect('Main_DB')

# Create a cursor object to interact with the database
cursor = conn.cursor()

# Your SQL query
create_values_df = '''
SELECT * from solar
'''

# Read SQL query result into a DataFrame
solar = pd.read_sql_query(create_values_df, conn)

# Close the connection
conn.close()

# Display the DataFrame
solar

,value,day,hour
0,0.0,2019-01-01,00:00
1,0.0,2019-01-01,01:00
2,0.0,2019-01-01,02:00
3,0.0,2019-01-01,03:00
4,0.0,2019-01-01,04:00
...,...,...,...
35063,0.0,2022-12-31,20:00
35064,0.0,2022-12-31,21:00
35065,0.0,2022-12-31,22:00
35066,0.0,2022-12-31,23:00


In [ ]:
import sqlite3
import pandas as pd

# Connect to the SQLite database (if the database doesn't exist, it will be created)
conn = sqlite3.connect('Main_DB')

# Create a cursor object to interact with the database
cursor = conn.cursor()

# Your SQL query
temp_df = '''
SELECT * from wind
'''

# Read SQL query result into a DataFrame
wind = pd.read_sql_query(temp_df, conn)
wind.rename(columns={'value': 'wind_generation'}, inplace=True)

# Your SQL query
temp_df = '''
SELECT * from spot_price
'''

# Read SQL query result into a DataFrame
spot_price = pd.read_sql_query(temp_df, conn)

# Read SQL query result into a DataFrame
spot_price = pd.read_sql_query(temp_df, conn)
spot_price.rename(columns={'value': 'spot_price'}, inplace=True)

# Your SQL query
temp_df = '''
SELECT * from solar
'''

# Read SQL query result into a DataFrame
solar = pd.read_sql_query(temp_df, conn)
solar.rename(columns={'value': 'solar_generation'}, inplace=True)

# Your SQL query
temp_df = '''
SELECT * from demand
'''

# Read SQL query result into a DataFrame
demand = pd.read_sql_query(temp_df, conn)
demand.rename(columns={'value': 'demand'}, inplace=True)

# SQL query
temp_df = '''
SELECT * from gas_data
'''

# Read SQL query result into a DataFrame
gas_data = pd.read_sql_query(temp_df, conn)
gas_data.rename(columns={'value': 'gas_price'}, inplace=True)
gas_data_hourly = gas_data.reindex(gas_data.index.repeat(24)).reset_index(drop=True)

# Close the connection
conn.close()

# Display the DataFrame


In [ ]:
df=wind.merge(spot_price, left_on=['day', 'hour'], right_on=['day', 'hour']) \
        .merge(solar, left_on=['day', 'hour'], right_on=['day', 'hour']) \
        .merge(demand, left_on=['day', 'hour'], right_on=['day', 'hour']) \
        .merge(gas_data, left_on=['day'], right_on=['delivery_day'])\
        .drop('delivery_day', axis=1)

df

,wind_generation,day,hour,spot_price,solar_generation,demand,index_price
0,3214.0,2019-01-01,00:00,66.88,0.0,23753.0,24.45
1,3222.0,2019-01-01,01:00,66.88,0.0,23018.0,24.45
2,3081.0,2019-01-01,02:00,66.00,0.0,21808.0,24.45
3,3069.0,2019-01-01,03:00,63.64,0.0,20635.0,24.45
4,2973.0,2019-01-01,04:00,58.85,0.0,19824.0,24.45
...,...,...,...,...,...,...,...
50948,10402.5,2022-12-31,22:00,1.88,0.0,21761.5,57.91
50949,10402.5,2022-12-31,22:00,1.88,0.0,21761.5,59.67
50950,10305.0,2022-12-31,23:00,0.10,0.0,20590.3,57.91
50951,10305.0,2022-12-31,23:00,0.10,0.0,20590.3,59.67


In [ ]:
from datetime import datetime, timedelta

# Given timestamp
given_timestamp = "2019-01-01T05:00:00.000+01:00"

# Convert the given timestamp string to a datetime object
timestamp_datetime = datetime.fromisoformat(given_timestamp)

# Extract the timezone offset from the timestamp
timezone_offset = timestamp_datetime.utcoffset()

# Add the timezone offset to adjust the hour
adjusted_datetime = timestamp_datetime + timezone_offset

# Format the adjusted datetime into the desired format
formatted_timestamp = adjusted_datetime.strftime("%Y-%m-%dT%H:%M")

print(formatted_timestamp)  # Output: 2019-01-01T06:00


2019-01-01T06:00


In [ ]:
from datetime import datetime, timedelta
import pandas as pd

# Sample DataFrame
data = {
    'timestamps': [
        "2019-01-01T05:00:00.000+01:00",
        "2020-02-02T07:00:00.000+02:00",
        "2020-02-02T07:00:00.000+01:00",
        # Add more timestamps as needed
    ]
}

df = pd.DataFrame(data)
'''
# Function to adjust timestamps based on their timezone offsets
def adjust_timestamp(row):
    timestamp_str, offset_str = row.split('+')
    timestamp_datetime = datetime.fromisoformat(timestamp_str)
    offset_hours = int(offset_str[:2])  # Extract hours from the offset
    offset_minutes = int(offset_str[-2:])  # Extract minutes from the offset
    timezone_offset = timedelta(hours=offset_hours, minutes=offset_minutes)
    adjusted_datetime = timestamp_datetime + timezone_offset
    return adjusted_datetime.strftime("%Y-%m-%dT%H:%M")

# Apply the adjustment function to the DataFrame column
df['adjusted_timestamps'] = df['timestamps'].apply(adjust_timestamp)
'''
print(df)


                      timestamps
0  2019-01-01T05:00:00.000+01:00
1  2020-02-02T07:00:00.000+02:00
2  2020-02-02T07:00:00.000+01:00


In [ ]:
# divide and transform to datetime
df[['timestamp_str', 'offset_str']] = df['timestamps'].str.split('+', expand=True)
df['timestamp_str'] = pd.to_datetime(df['timestamp_str'])

# Get offset
df['hour_offset'] = df['offset_str'].astype(str).str[0:2]
df['minute_offset'] = df['offset_str'].astype(str).str[3:5]

# Convert hour and minute offsets to timedelta objects
df['timezone_offset'] = pd.to_timedelta(df['hour_offset'].astype(int), unit='h') + pd.to_timedelta(df['minute_offset'].astype(int), unit='m')
df['adjusted_datetime'] = df['timestamp_str'] + df['timezone_offset']

# Keep only desired columns

df

,timestamps,timestamp_str,offset_str
0,2019-01-01T05:00:00.000+01:00,2019-01-01 05:00:00,01:00
1,2020-02-02T07:00:00.000+02:00,2020-02-02 07:00:00,02:00
2,2020-02-02T07:00:00.000+01:00,2020-02-02 07:00:00,01:00


In [ ]:
df['hour_offset'] = df['offset_str'].astype(str).str[0:2]
df['minute_offset'] = df['offset_str'].astype(str).str[3:5]
# Assuming 'hour_offset' and 'minute_offset' are columns representing hour and minute offsets, respectively

# Convert hour and minute offsets to timedelta objects
df['timezone_offset'] = pd.to_timedelta(df['hour_offset'].astype(int), unit='h') + pd.to_timedelta(df['minute_offset'].astype(int), unit='m')

df

,timestamps,timestamp_str,offset_str,hour_offset,minute_offset,timezone_offset
0,2019-01-01T05:00:00.000+01:00,2019-01-01 05:00:00,01:00,01,00,0 days 01:00:00
1,2020-02-02T07:00:00.000+02:00,2020-02-02 07:00:00,02:00,02,00,0 days 02:00:00
2,2020-02-02T07:00:00.000+01:00,2020-02-02 07:00:00,01:00,01,00,0 days 01:00:00


In [ ]:
df['adjusted_datetime'] = df['timestamp_str'] + df['timezone_offset']
df

,timestamps,timestamp_str,offset_str,hour_offset,minute_offset,timezone_offset,adjusted_datetime
0,2019-01-01T05:00:00.000+01:00,2019-01-01 05:00:00,01:00,01,00,0 days 01:00:00,2019-01-01 06:00:00
1,2020-02-02T07:00:00.000+02:00,2020-02-02 07:00:00,02:00,02,00,0 days 02:00:00,2020-02-02 09:00:00
2,2020-02-02T07:00:00.000+01:00,2020-02-02 07:00:00,01:00,01,00,0 days 01:00:00,2020-02-02 08:00:00


f

In [ ]:

from ESIOS import ESIOS

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from ESIOS import ESIOS
from mibgas import mibgas

# load data from ESIOS since 2019

start_date = "2019-01-01T00:00:00"
end_date = "2019-03-01T00:00:00"

#training_data = ['demand', 'wind', 'solar', 'spot_price']
training_data = ['nuclear']
# convert start_date and end_date to datetime objects
start_date = datetime.fromisoformat(start_date)
end_date = datetime.fromisoformat(end_date)
current_date = start_date

while current_date <= end_date:
    # calculate the end of the current month
    
    if current_date + relativedelta(years=1)>end_date:
        query_year = end_date
    else:
        query_year = current_date+ relativedelta(years=1)
        
        
        # format end_of_month in the "%Y-%m-%dT%H:%M:%S" format
    query_year_formated = query_year.strftime("%Y-%m-%dT%H:%M:%S")
    current_date_formated = current_date.strftime("%Y-%m-%dT%H:%M:%S")

    for item in training_data:
        try:
            generation_data = ESIOS.get_training_data(current_date_formated, query_year_formated, item)
            df = ESIOS.extract_values(generation_data, item)

            ESIOS.connection_to_DB(df, item, current_date_formated)
        except Exception as e:
            print(f'Values were not found for {item} for the month{current_date_formated}')
            print(e)
    
    current_date = current_date + relativedelta(years=1)

'''
# load data from MIBGAS since 2019
    
years = [2019, 2020, 2021, 2022, 2023]

for year in years:
    gas_data = mibgas.get_data(year)
    gas_data_cleaned = mibgas.adapt_data(gas_data, year)
    mibgas.connection_to_DB(gas_data_cleaned, year)
    '''

5577 nuclear data values uploaded succesfully for year 2019 


'\n# load data from MIBGAS since 2019\n    \nyears = [2019, 2020, 2021, 2022, 2023]\n\nfor year in years:\n    gas_data = mibgas.get_data(year)\n    gas_data_cleaned = mibgas.adapt_data(gas_data, year)\n    mibgas.connection_to_DB(gas_data_cleaned, year)\n    '

In [1]:
import os
import sys
print(os.path.dirname(sys.executable))

c:\Program Files\Python311
